In [ ]:
# importing modules
import pandas as pd
import os
from datetime import date
from datetime import datetime
import time
import pytz
from ast import Name
from google.colab import drive

In [ ]:
# Mount google drive
drive.mount('/content/drive')

# Folder path containing the Excel files
folder_path = '/content/blue'

# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xlsx") or file_name.endswith(".xls"):

        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Read the Excel file
        df = pd.read_excel(file_path, header = 1)

        # Add a "filename" column with a specific value based on the file title
        df["Filename"] = file_name

        # Rename columns
        df.rename(columns={'Resource Type': 'EMResource Region',
                       'Resource': 'Facility Name',}, inplace=True)


 ############### time related coding ###############

        # saving or exporting files to current date
        current_date = str(time.strftime("%Y-%m-%d %H:%M:%S"))

        # Add "date event started" and "date event ended" columns
        df["Date Event Started"] = '08/04/2023 14:04' ##### hardcoded, need to change #####
        df["Date Event Ended"] = ''

        # Get the current UTC time
        utc_time = datetime.utcnow()

        # Define the time zone for Colorado (Mountain Time Zone)
        colorado_tz = pytz.timezone('America/Denver')

        # Convert the UTC time to Colorado time zone
        colorado_time = utc_time.replace(tzinfo=pytz.utc).astimezone(colorado_tz)

        # Format the time in Colorado's timezone
        df['Date Collected'] = colorado_time.strftime('%Y-%m-%d %H:%M:%S %Z')
        df['Time'] = colorado_time.strftime('%H:%M:%S %Z')

############## merging files together ###############

        # Append the processed data to the merged DataFrame
        merged_data = merged_data.append(df)

############### adding license ID to file ###############

# read file containing the license ID information for each facility
path = '/content/drive/MyDrive/facilities.xlsx'
path1 = pd.read_excel (path)

# drop unneeded columns in the datafile
drop_col = ['Type', 'EMResource ID', 'AHA ID', 'External ID', 'Website', 'Contact',
       'Phone 1', 'Phone 2', 'Fax', 'Email', 'Notes']

idno = path1.drop(columns = drop_col)

# Rename columns
idno.rename(columns={'Resource Name' : 'EMResource Region',
                   'License #  (State will populate)': 'License ID'}, inplace=True)

# Merging license ID # data to merged data
merge_column = 'EMResource Region'
result_df = merged_data.merge (idno, on=merge_column)

# Save the merged DataFrame to a new Excel file
# output_file_path = os.path.join(folder_path, 'merged_license_%s.xlsx' % current_date)
output_file_path = '/content/merged_license.xlsx'
result_df.to_excel(output_file_path, index=False)